In [2]:
from curry import curry
from inspect import signature
from lambda_calculus import *
from recursive import recursive
from chain import LightChain as Chain
from typing import Callable, TypeVar, ParamSpec, NewType, Any
P, Q = ParamSpec('P'), ParamSpec('P')
T, _S = TypeVar('T'), TypeVar('S')
F = Callable[[T],T]
Recusive = int
Functor = Callable[[F],F]
Pair = Callable[P,Callable[Q,Callable[[Callable[P,Callable[Q,T]]],T]]]

In [2]:
@curry
def add(a, b):
    return a + b

@curry
def mod(a, b):
    return b % a

In [3]:
init = value(0)
init(identify)

0

In [4]:
print(extract(init))
print(extract(recursive(3)))

0
3


In [5]:
inc = lambda g: lambda h: h(g(lambda x: x + 1))
extract(inc(inc(init)))

2

In [6]:
oneNtwo = pair(1)(2)
print(first(oneNtwo), second(oneNtwo))

1 2


In [7]:
extract(pred(5)(inc)(init))

4

In [65]:
def print_return(arg):
    print(arg)
    return arg

In [90]:
class Category:
    value:Callable[[T], Callable[[Callable], T]] = lambda v: (lambda h: h(v))
    const:Callable[[T], Callable[[Callable], T]] = lambda v: lambda u: v
    inc:Callable[[F], Callable[[Callable], T]] = lambda f: lambda g: lambda h: h(g(f))
    succ:Callable[[Recusive], Callable[[F], T]] \
        = lambda f: lambda x: lambda n: f(recursive(n)(f)(x))
    
    def __init__(self, f:F, x:T):
        self.__f = f
        self.const = Category.const(x)
        self.init = Category.value(x)
        self.inc = Category.inc(f)
        self.succ = Category.succ(self.inc)(self.init)
        self.plus:Callable[[Recusive], Callable[[Recusive], T]] \
            = lambda m: lambda n: recursive(m)(self.inc)(recursive(n)(self.inc)(self.init))
        self.curr_succ:Callable[[Pair], Pair] = lambda p: pair(second(p))(self.inc(second(p)))
        self.pred:Callable[[Recusive], T] \
            = lambda n: first(recursive(print_return(n))(self.curr_succ)(pair(self.init)(self.init)))
        self.pred_:Callable[[Recusive], T] \
            = lambda n: recursive(n)(self.inc)(lambda u: x)(lambda u: u)
        self.minus:Callable[[Recusive], Callable[[Recusive], Functor]]\
            = lambda m: lambda n: (recursive(n)(self.pred))


def check(c:Category, name:str) -> None:
    print("Category:", name)
    print("init:", extract(c.init))
    print("inc inc:", extract(c.inc(c.inc(c.init))))
    print("succ(2):", extract(c.succ(2)))
    print("plus(2)(3):", extract(c.plus(2)(3)))
    print("pred(2):", c.pred_(2))
    # print("minus(2)(5):", extract(c.minus(2)(5)))
    print()


N = Category(add(1), 0)
check(N, "Natural Number")
E = Category(add(2), 0)
check(E, "Even Number")
O = Category(add(2), 1)
check(O, "Odd Number")
S = Category("f( {})".format, "x")
check(S, "String")

Category: Natural Number
init: 0
inc inc: 2
succ(2): 3
plus(2)(3): 5
pred(2): 1

Category: Even Number
init: 0
inc inc: 4
succ(2): 6
plus(2)(3): 10
pred(2): 2

Category: Odd Number
init: 1
inc inc: 5
succ(2): 7
plus(2)(3): 11
pred(2): 3

Category: String
init: x
inc inc: f( f( x))
succ(2): f( f( f( x)))
plus(2)(3): f( f( f( f( f( x)))))
pred(2): f( x)



In [ ]:
f = lambda p: pair(second(p))(succ(1)(inc)(second(p)))
zero = lambda f: lambda x: x
pc0 = pair(zero)(zero)
pred = lambda n: first(n(f)(pc0))
extract(pred(recursive(3)))

In [ ]:
print(extract(pred(pred(3))))

In [ ]:
minus = lambda m: lambda n: recursive(n)((pred)(m))
print(extract(minus(7)(3)))